In [52]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from itertools import combinations, product
import time
import ssl
import os
import re
from tqdm.notebook import tqdm
import random

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select

In [53]:
#For ignoring SSL ceritficate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [54]:
class infine_scroll(object): 
  def __init__(self, last):
    self.last = last

  def __call__(self, driver):
    new = driver.execute_script('return document.body.scrollHeight')  
    if new > self.last:
        return new
    else:
        return False

In [55]:
def get_html_from_url(url):
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver') 
  browser.set_page_load_timeout(30) 
  browser.get(url)
   
  last_height = browser.execute_script('return document.body.scrollHeight')

  flag=1

  while flag==1:
    
    try:
       browser.execute_script('window.scrollTo(0,document.body.scrollHeight)')
       wait = WebDriverWait(browser, 10)

       new_height = wait.until(infine_scroll(last_height))
       last_height = new_height

    except:
        print("End of page reached")
        flag = 0
  try:
    html = browser.page_source
  except:
    print('something wrong with ' + url)
    html = None
  return html

In [56]:
df_path = r"C:\Users\Ravit\Documents\rnd\horizon_scanning_lab\Scrapers\Quora_RCS\answers_and_info\bert_outcome_analysis\first_iteration\words_in_clusters_manual_cleaning.csv"
df_sentences = pd.read_csv(df_path)

In [57]:
df_sentences.set_index(['Unnamed: 0'], inplace=True)

In [58]:
df_sentences

,old_list,new_list,comments
Unnamed: 0,,,
-1,company solar cryptocurrency,solar cryptocurrency,NaN
0,fusion nuclear plasma power reactors reactor f...,fusion nuclear plasma power reactor fission fuel,NaN
1,brain lobe body jess cells function parts driv...,brain lobe body cells function parts driver,NaN
2,50 super space computing,super space computing,NaN
3,3d emerging printers printing list 10 reality ...,3d emerging printers printing reality ai trends,NaN
4,iot internet things devices connected billion ...,iot internet of things devices connected smart...,NaN
5,following reasonable 2031 2068 vaguely thingsa...,NaN,empty content
6,predictions cars electric cp group futurists t...,predictions cars electric cp_group futurists ...,NaN
7,students communication issue letter cover busi...,students communication issue business informa...,NaN


In [59]:
df_sentences = df_sentences['new_list'].to_frame()

In [60]:
df_sentences

,new_list
Unnamed: 0,
-1,solar cryptocurrency
0,fusion nuclear plasma power reactor fission fuel
1,brain lobe body cells function parts driver
2,super space computing
3,3d emerging printers printing reality ai trends
4,iot internet of things devices connected smart...
5,NaN
6,predictions cars electric cp_group futurists ...
7,students communication issue business informa...


In [61]:
df_sentences.dropna(axis=0, inplace=True)

In [62]:
df_sentences

,new_list
Unnamed: 0,
-1,solar cryptocurrency
0,fusion nuclear plasma power reactor fission fuel
1,brain lobe body cells function parts driver
2,super space computing
3,3d emerging printers printing reality ai trends
4,iot internet of things devices connected smart...
6,predictions cars electric cp_group futurists ...
7,students communication issue business informa...
8,computers desktops road wrist keyboard


In [63]:


search_link = 'https://www.quora.com/search?q='

#create search links
all_search_links = []
for i, row in tqdm(df_sentences.iterrows()):
    sentence_words = row.iloc[0].split()
    for i, word in enumerate(sentence_words):
        if '_' in word:
            word = word.replace('_', ' ')
            sentence_words[i] = word
    
    all_search_links.append(search_link+'%20'.join(sentence_words))
    if len(sentence_words)==2:
        sentence_words = sentence_words[::-1]
    else:
        #shuffle sentence to get different results
        random.shuffle(sentence_words)
    all_search_links.append(search_link+'%20'.join(sentence_words))

0it [00:00, ?it/s]

In [64]:
all_search_links

['https://www.quora.com/search?q=solar%20cryptocurrency',
 'https://www.quora.com/search?q=cryptocurrency%20solar',
 'https://www.quora.com/search?q=fusion%20nuclear%20plasma%20power%20reactor%20fission%20fuel',
 'https://www.quora.com/search?q=fusion%20power%20reactor%20plasma%20fuel%20fission%20nuclear',
 'https://www.quora.com/search?q=brain%20lobe%20body%20cells%20function%20parts%20driver',
 'https://www.quora.com/search?q=driver%20parts%20lobe%20function%20body%20cells%20brain',
 'https://www.quora.com/search?q=super%20space%20computing',
 'https://www.quora.com/search?q=space%20super%20computing',
 'https://www.quora.com/search?q=3d%20emerging%20printers%20printing%20reality%20ai%20trends',
 'https://www.quora.com/search?q=printers%203d%20reality%20trends%20ai%20emerging%20printing',
 'https://www.quora.com/search?q=iot%20internet%20of%20things%20devices%20connected%20smart%20network%20home%20embedded',
 'https://www.quora.com/search?q=home%20connected%20embedded%20network%20sma

In [65]:
len(all_search_links)

32

In [68]:
class Extract_href_urls():
    def __init__(self, links):
        self.links = links
        self.all_box_links = []
        self.all_indexes = []
        self.get_question_links()
        self.all_hrefs = []        
        self.get_hrefs()
          
    def get_question_links(self):    
        for j, link in enumerate(self.links):
            url_html = get_html_from_url(link)
            if url_html is None:
                continue
            soup = BeautifulSoup(url_html)
            #find all boxes of groups
            self.all_box_l = soup.find_all('a', {'class':"q-box qu-display--block qu-cursor--pointer qu-hover--textDecoration--underline Link___StyledBox-t2xg9c-0 dxHfBI"})
            try:
                self.all_indexes.extend([df_sentences.index[j]]*len(self.all_box_l))
                self.all_box_links.extend(self.all_box_l)
            except:
                pass
            

    @staticmethod
    def get_url_tag(box):
        #By find_element not working in selenium using string parsing instead
        url=None
        box = str(box)
        split_box = box.split()
        for word in split_box:
            if 'href' in word:
                url = word.strip('href=').strip('"')
                break
        return url
    
    def get_hrefs(self): 
        for box in self.all_box_links:
            link_hrefs = self.get_url_tag(box)
            self.all_hrefs.append(link_hrefs)

In [69]:
res1 = Extract_href_urls(all_search_links)

C:\Users\Ravit\AppData\Local\Temp\ipykernel_17676\1869657974.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome('C:\Program Files\chromedriver_win32 (1)\chromedriver')


End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached
End of page reached


In [71]:
df_res = pd.DataFrame(res1.all_hrefs)

In [74]:
df_res['cluster_index'] = res1.all_indexes

In [76]:
len(df_res)

2467

In [79]:
df_res.drop_duplicates()

,0,cluster_index
0,https://www.quora.com/What-would-happen-to-cry...,-1
1,https://www.quora.com/What-solar-panels-would-...,-1
2,https://www.quora.com/Is-Cryptocurrency-not-ge...,-1
3,https://www.quora.com/Would-mining-cryptocurre...,-1
4,https://www.quora.com/If-I-have-free-electrici...,-1
...,...,...
2462,https://www.quora.com/How-can-a-politics-stude...,17
2463,https://www.quora.com/How-should-I-prepare-for...,17
2464,https://www.quora.com/If-Donald-Trump-is-not-r...,17
2465,https://www.quora.com/How-can-you-improve-your...,17


In [80]:
save_path = r'C:\Users\Ravit\Documents\rnd\horizon_scanning_lab\Scrapers\Quora_RCS\answers_and_info\bert_outcome_analysis\second_iteration\links_for_second_quora_scrap.csv'
df_res.to_csv(save_path, index=False)

In [81]:
set(df_res['cluster_index'])

{-1, 0, 1, 2, 3, 4, 6, 7, 8, 10, 11, 12, 16, 17}

In [82]:
set(df_sentences.index) - set(df_res['cluster_index'])

{13, 15}